In [1]:
import os
import matplotlib.pyplot as plt
from dataset import dataset
from wdsr import wdsr
from train import WdsrTrainer
from common import resolve
from common import resolve_single
from common import evaluate

In [2]:
# set depth for res blocks and scale (always 4)

depth = 32
scale = 4
features = 32

In [3]:
# set folder to save weight into 

weights_dir = './weights/wdsr/'
weights_file = os.path.join(weights_dir, 'weightsB{}F{}.h5'.format(depth, features))
os.makedirs(weights_dir, exist_ok=True)

In [4]:
# create our training and validation datasets 
# note, strange warning caused by using tfa2.1 with tf2.2 (safe to ignore... I think...)

train_ds = dataset(batch_size=16, random_transform=True, subset='train')
valid_ds = dataset(batch_size=1, repeat_count=1, random_transform=False, subset='valid')

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on

In [5]:
model=wdsr(scale=scale, num_filters=features, num_res_blocks=depth, res_block_scaling=0.1)
model.load_weights('weights/wdsr/weights.h5')

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [9]:
# initialize the trainer

trainer = WdsrTrainer(model=model, loss='MAE', checkpoint_dir=f'.ckpt/wdsr-{depth}-x{scale}')

In [ ]:
# train!

trainer.train(train_ds, valid_ds.take(10), steps=60000, evaluate_every=100, save_best_only=True)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
